### Example: Fibonacci Sequence

In [10]:
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [11]:
[fib_recursive(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

But this quickly becomes an issue as `n` grows larger:

In [12]:
from timeit import timeit

In [13]:
timeit('fib_recursive(10)', globals=globals(), number=10)

0.00020240000000626424

In [14]:
timeit('fib_recursive(28)', globals=globals(), number=10)

1.2346476000000166

In [15]:
timeit('fib_recursive(29)', globals=globals(), number=10)

2.0710665999999947

We can alleviate this by using memoization:

In [16]:
from functools import lru_cache

In [17]:
@lru_cache()
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [18]:
timeit('fib_recursive(10)', globals=globals(), number=10)

8.800000017572529e-06

In [19]:
timeit('fib_recursive(29)', globals=globals(), number=10)

1.3600000016822378e-05

In [20]:
@lru_cache()
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [21]:
fib_recursive(2000)

RecursionError: maximum recursion depth exceeded

So we can use a non-recursive approach to calculate the `n-th` Fibonacci number:

In [22]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1

In [23]:
[fib(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

This works well for large `n` values too:

In [24]:
timeit('fib(5000)', globals=globals(), number=10)

0.005792499999984102

In [25]:
class Fib:
    def __init__(self, n):
        self.n = n
        
    def __iter__(self):
        return self.FibIter(self.n)
        
    class FibIter:
        def __init__(self, n):
            self.n = n
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.n:
                raise StopIteration
            else:
                result = fib(self.i)
                self.i += 1
                return result

And we can now iterate the usual way:

In [26]:
fib_iterable = Fib(7)

In [27]:
for num in fib_iterable:
    print(num)

1
1
2
3
5
8
13


Of course, we can also use the second form of the `iter` function too, but we have to create a closure first:

In [28]:
def fib_closure():
    i = 0
    def inner():
        nonlocal i
        result = fib(i)
        i += 1
        return result
    return inner

In [29]:
fib_numbers = fib_closure()
fib_iter = iter(fib_numbers, fib(7))
for num in fib_iter:
    print(num)

1
1
2
3
5
8
13


But there's two things here:

1. The syntax for either implementation is a little convoluted and not very clear
2. More importantly, notice what happens every time the `next` method is called - it has to calculate every Fibonacci number from scratch (using the `fib` function) - that is wasteful...

Instead, we can use a generator function very effectively here.

Here is our original `fib` function:

In [30]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1    

In [31]:
[fib(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

Now let's modity it into a generator function:

In [32]:
def fib_gen(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

In [33]:
[num for num in fib_gen(7)]

[2, 3, 5, 8, 13, 21]

We're almost there. We're missing the first two Fibonacci numbers in the sequence - we need to yield those too.

In [34]:
def fib_gen(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

In [35]:
[num for num in fib_gen(7)]

[1, 1, 2, 3, 5, 8, 13, 21]

And finally we're returning one number too many if `n` is meant to indicate the length of the sequence:

In [36]:
def fib_gen(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for i in range(n-2):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

And now everything works fine:

In [37]:
[num for num in fib_gen(7)]

[1, 1, 2, 3, 5, 8, 13]

Let's time it as well to compare it with the other methods:

In [38]:
timeit('[num for num in Fib(5_000)]', globals=globals(), number=1)

1.3445508000000075

In [39]:
fib_numbers = fib_closure()
sentinel = fib(5_001)

timeit('[num for num in iter(fib_numbers, sentinel)]', globals=globals(),
      number=1)

1.3417884000000129

In [40]:
timeit('[num for num in fib_gen(5_000)]', globals=globals(), number=1)

0.0016880999999955293